In [1]:
import xarray as xr
import numpy as np
import netCDF4 as nc
import os
import metpy.calc as mpcalc
from metpy.units import units

ncpath='/fs/ess/PFS0220/eurec4a/case_feb9th_144_100m_2/eurec4a.default.0000000.nc'
nc_out_path = '/fs/ess/PFS0220/eurec4a/microhh_results/eurec4a_feb9_512_200m_30kmtop_1.nc'


microhh_out=xr.open_dataset(ncpath,decode_times=True)
rootgroup = nc.Dataset(ncpath,"r")
for group in rootgroup.groups:
    microhh_out=xr.merge([microhh_out, xr.open_dataset(ncpath,group=group,decode_times=True)])

def rename_vars(ds):
    ds['ps']=ds['phydroh'].isel(zh=0)
    ds['ts']=ds['T'].isel(z=0)
    ds['wspd']=ds['u'].isel(z=0)
    for i in range(1,ds['time'].size):
        ds['ts'][i]=mpcalc.temperature_from_potential_temperature(ds['ps'].isel(time=i),ds['thl_bot'].isel(time=i))
        ds['wspd'][i]=mpcalc.wind_speed(ds['u'].isel(z=0,time=i),ds['v'].isel(z=0,time=i))
    ds['pr']=ds['rr']
    ds['qv']=ds['qt']
    ds['qv']=ds['qt']-ds['ql']
    ds['rlds']=ds['lw_flux_dn'].isel(zh=0)
    ds['rlus']=ds['lw_flux_up'].isel(zh=0)
    #ds=ds.rename_dims({'z':'zf'})
    ds=ds.rename({'z':'zf','obuk':'ol','ql_path':'lwp','ql_frac':'flc','qi_path':'iwp','p':'pa','u':'ua','v':'va','rho':'rhoa','T':'ta','rh':'hur','ql':'qlc','qr':'qlr','qi':'qic','thl':'thetal'})
    #ds=ds.rename({'xu':'x','yu':'y','zu':'z'})
    ds['hfss']=ds['rhoa'].isel(zf=0)*1004*ds['thl_flux'].isel(zh=0)
    ds['hfls']=ds['rhoa'].isel(zf=0)*2500000*ds['qt_flux'].isel(zh=0)
    return ds

microhh_out=rename_vars(microhh_out)
if os.path.exists(nc_out_path):
    os.remove(nc_out_path)
microhh_out.to_netcdf(nc_out_path)